In [6]:
import numpy as np 
import sympy
import scipy


Setting up a function to create the density matrix $\hat\rho$. Given a numer of levels, this function returns a matrix populated with Sympy symbolic objetcs wich will allow us to set up the equations and manipulate the terms.

In [7]:
def create_rho_list(levels = 3):
    rho_list = []
    for i in range(levels):
        for j in range(levels):
            rho_list.append(sympy.Symbol('rho_'+str(i+1)+str(j+1)))
    return rho_list

In [8]:
print(create_rho_list())

[rho_11, rho_12, rho_13, rho_21, rho_22, rho_23, rho_31, rho_32, rho_33]


In [9]:
def create_rho_matrix(levels=3):
    rho_matrix = np.empty((levels, levels), dtype = 'object')
    for i in range(levels):
        for j in range(levels):
            rho_matrix[i,j]=(sympy.Symbol('rho_'+str(i+1)+str(j+1)))
            
    return np.matrix(rho_matrix)

In [10]:
print(create_rho_matrix())

[[rho_11 rho_12 rho_13]
 [rho_21 rho_22 rho_23]
 [rho_31 rho_32 rho_33]]


Setting up the Hamiltonian for any n-level ladder system where each level is only coupled to the levels directle above and velow it (the ladder system).


In [17]:
def Hamiltonian(Omegas, Deltas):
    levels = len(Omegas)+1
    H = np.zeros((levels, levels))
    for i in range(levels):
        for j in range(levels):
            if i==j and i!=0:
                H[i,j] = -2*(np.sum(Deltas[:i]))
            elif np.abs(i-j) == 1:
                H[i,j] = Omegas[np.min([i,j])]
                
    return np.matrix(H/2)

In [18]:
Omegas = [1,3]
Deltas = [2,4]
H = Hamiltonian(Omegas, Deltas)
print(H)

[[ 0.   0.5  0. ]
 [ 0.5 -2.   1.5]
 [ 0.   1.5 -6. ]]


### Decay and Dephasing 
Next we set up the matrix form of the decay operator $\hat{\mathcal{L}}$. The decay operator will be split into two separate parts for simplicty: the atomic decay $\hat{L}_{\rm{atom}}$ and the dephasing due to the finite linewidths of the driving fields $\hat{L}_{\rm{dephasing}}$. The total operator will then be the sum of these parts $\hat{\mathcal{L}} = \hat{L}_{\rm{atom}} + \hat{L}_{\rm{dephasing}}$.

##### Linblad superoperator $\hat{L}_{\rm{atom}}$ describing spontaneous decay:
Assuming that each level can only decay to the level below it at a rate $\Gamma_n$, and that there is no decay out of the bottom level, the coherences (off-diagonal elements) will be as $L_{ij,\, i\neq j} = -\frac{\Gamma_i + \Gamma_j}{2}\rho_{ij}$ while the populations (diagonal elements) will be given by $L_{ij, \, i=j} = \Gamma_{i+1}\rho_{i+1, j+1} - \Gamma_{i}\rho_{ij}$. This means that for a 3-level system we have

$$\hat{L}_{\rm{atom}} = \begin{pmatrix} 
\Gamma_2 \rho_{22} - \Gamma_1\rho_{11} & - \frac{\Gamma_1 + \Gamma_2}{2}\rho_{12} & -\frac{\Gamma_1 + \Gamma_3}{2}\rho_{13} \\ 
-\frac{\Gamma_2 + \Gamma_1}{2}\rho_{21} & \Gamma_3 \rho_{33} - \Gamma_2\rho_{22} & -\frac{\Gamma_2 + \Gamma_3}{2}\rho_{23} \\
-\frac{\Gamma_3 + \Gamma_1}{2}\rho_{31} & -\frac{\Gamma_3 + \Gamma_2}{2}\rho_{32} &  - \Gamma_3\rho_{33} \end{pmatrix}$$
Making the assumption that there is no decay out of the lower level ($\Gamma_1 = 0$) simplifies the expression to 
$$\hat{L}_{\rm{atom}} = \begin{pmatrix} 
\Gamma_2 \rho_{22} & - \frac{\Gamma_2}{2}\rho_{12} & -\frac{\Gamma_3}{2}\rho_{13} \\ 
-\frac{\Gamma_2}{2}\rho_{21} & \Gamma_3 \rho_{33} - \Gamma_2\rho_{22} & -\frac{\Gamma_2 + \Gamma_3}{2}\rho_{23} \\
-\frac{\Gamma_3}{2}\rho_{31} & -\frac{\Gamma_3 + \Gamma_2}{2}\rho_{32} &  - \Gamma_3\rho_{33} \end{pmatrix}$$
